In [87]:
import pandas as pd

In [88]:
def chunk_class(tag):
    
    """
        Converte as classes Spacy em um modelo customizado...
    """
    
    final = tag
    
    try:
        pre, suf = tag.split('-')
    except:
        return final
    
    if suf == 'PERSON': 
        final = pre + '-PER'
        
    if suf == 'EVENT':
        final = pre + '-EVENT'
        
    if suf == 'GPE' or suf == 'FAC': 
        final = pre + '-LOC'
        
    if suf == 'ORG':
        final = pre + '-ORG'
        
    return final

In [89]:
data1 = pd.read_csv('ANOTA_1.csv')
data2 = pd.read_csv('ANOTA_2.csv')

In [90]:
data1.Tag = data1.Tag.apply(chunk_class)
data2.Tag = data2.Tag.apply(chunk_class)

In [135]:
from tqdm import tqdm

def reduce_ner_class(candidates_preds: zip) -> list:

    if not isinstance(candidates_preds, zip):
        raise Exception("Tipo de estrutura não é aceita!")
        
    final_tags = []

    for tags in tqdm(candidates_preds):
        if not 'O' in tags:
            tag = max(tags)
            final_tags.append(tag)
        else:
            tag = tuple(filter((lambda x: x!= 'O'), tags))

            if tag:
                final_tags.append(tag[0])
            else:
                final_tags.append('O')
    
    return final_tags

In [136]:
new_pd = pd.DataFrame()
new_pd['Word'] = data1.Word
new_pd['Tag'] = reduce_ner_class(zip(data1.Tag.values, data2.Tag.values))

52it [00:00, 26996.39it/s]


In [137]:
new_pd

,Word,Tag
0,We,O
1,could,O
2,go,O
3,two,B-CARDINAL
4,ways,O
5,with,O
6,this,O
7,one,O
8,.,O
9,One,B-CARDINAL
